In [ ]:
DESCRIPTION = """A tool for sharing atomate2 workflows, which stores processing steps in this database and stores VASP runs in vaspingestor"""

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

fw_doc:
    type: Dict
    description: Atomate2 firework document
    value: {}

wf_id:
    type: Int
    description: Workflow id
    value: 0

wf_graph:
    type: Dict
    description: Atomate2 workflow graph
    value: {}
        
author:
    type: Tag
    description: Author to be associated with the workflow
    value: ""
        
dataset:
    type: Tag
    description: Dataset to be associated with the workflow
    value: ""

In [ ]:
EXTRA_FILES = []

In [ ]:
from simtool import getValidatedInputs

defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)

In [ ]:
from simtool import getParamsFromDictionary

inputParameters = getParamsFromDictionary(INPUTS,globals())

In [ ]:
%%yaml OUTPUTS

fw_id:
    type: Int
    description: Fireworks ID of the task
    value: 0

name:
    type: Str
    description: Name of the firework step
    value: ""

structure:
    type: Dict
    description: Pymatgen structure associated with the firework
    value: ""

vasp_id:
    type: Str
    description: ID returned from the vaspingestor tool, if ran
    value: ""

outputs:
    type: Dict
    description: Simplified set of outputs of the firework
    value: {}

In [ ]:
# Import required libraries
import numpy as np
from pymatgen.core import Structure


In [ ]:
# Validate inputs
if not fw_doc:
    raise Exception("No fw_doc provided")

# check if the @class key is JobStoreDocument
if fw_doc.get('@class') != 'JobStoreDocument':
    raise Exception("This doesn't appear to be a JobStoreDocument")


In [ ]:
# extract the outputs from the fw_doc
fw_id = fw_doc['metadata']['fw_id']
name = fw_doc['name']
structure = fw_doc['output'].get('structure', {})

In [ ]:
# Import nanoHUB-specific libraries
import nanohubremote as nr
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

In [ ]:
# if the fw_doc is a vasp run, store it in vaspingestor and get the vasp_id in return
if fw_doc['output'].get('run_type','') in ['PBE']: # TODO: add more run types

    # Load the Sim2L
    simToolName = "vaspingestor"
    simToolLocation = searchForSimTool(simToolName)
    installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
    
    # Get the Sim2L IO
    inputs = getSimToolInputs(simToolLocation)
    outputs = getSimToolOutputs(simToolLocation)

    ## Submit Sim2L sequentially
    # Set the inputs for the Sim2L
    inputs['doc'].value = fw_doc['output']
    inputs['author'].value = author
    inputs['dataset'].value = dataset

    # Run Sim2L
    r = Run(simToolLocation,inputs)
    
    # Obtain results for Sim2L
    r.getResultSummary()

In [ ]:
from simtool import DB
db = DB(OUTPUTS)

In [ ]:
db.save('fw_id',fw_id)
db.save('name',name)
db.save('structure',structure)
db.save('vasp_id',"")
db.save('outputs',{})